# Race Data Analysis

Get all xml tags of the data. Analyise which one are used in every datapoint which are used everwhere


In [1]:
import glob
import xml.etree.ElementTree as ET

alltkrptsextensions = []
alleverextensions = []
firstwrite = True

namespace = "{http://www.topografix.com/GPX/1/1}"
root_dir = '.\\GPX Files\\'

for filename in glob.iglob(root_dir + '**/*.gpx', recursive=True):
    #print ('currentFile', filename)
    root = ET.parse(filename).getroot()
    trknumber = 0
    for trk in root.findall(namespace + "trk"):
        if trknumber > 0:
            first_seg =  trk.find(namespace +"trkseg")
            first_trkpt = first_seg.find(namespace + "trkpt")
            for extensions in first_trkpt.findall(namespace + "extensions"):
                currentextensions = []
                for extension in extensions:
                    extension = str(extension).split(namespace)[1].split('\'')[0]
                    currentextensions.append(extension)
                    if extension not in alleverextensions:
                        alleverextensions.append(extension)
                if firstwrite:
                    alltkrptsextensions = currentextensions
                    firstwrite = False
                else:
                    for alltkrptsextension in alltkrptsextensions:
                        if alltkrptsextension not in currentextensions:
                            alltkrptsextensions.remove(alltkrptsextension)
        trknumber+=1
                        
print("consistent", alltkrptsextensions)
print(len(alltkrptsextensions))

print("all", alleverextensions)
print(len(alleverextensions))


consistent ['ts', 'dfs', 'h', 'v', 'atod', 'bpdr', 'dtf', 'dtfl', 'dtfr', 'dtr', 'dtrl', 'dtrr', 'epd', 'er', 'etd', 'odrn', 'sbc', 'sbv', 'ax', 'ay', 'vla', 'vlb', 'vsd', 'vyr', 'wtfl', 'wtfr', 'wtrl', 'wtrr', 'app', 'btd', 'drfl', 'drfr', 'drrl', 'drrr', 'ect', 'eot', 'iat', 'laig', 'lain', 'loig', 'loin', 'tg', 'tl', 'tsh', 'tot', 'ttfl', 'ttfr', 'ttrl', 'ttrr', 'ttsrl', 'tpfl', 'tpfr', 'tprl', 'tprr', 'tpms', 'tsfr', 'ac', 'swa', 'fwa']
59
all ['ts', 'dfs', 'h', 'v', 'da', 'atod', 'bpdr', 'dtf', 'dtfl', 'dtfr', 'dtr', 'dtrl', 'dtrr', 'ectdr', 'epd', 'er', 'etd', 'hvbs', 'odrn', 'sbc', 'sbv', 'ax', 'ay', 'vlfl', 'vlfr', 'vla', 'vlb', 'vlrl', 'vlrr', 'vs', 'vsd', 'vyr', 'wrfl', 'wrfr', 'wrrl', 'wrrr', 'wtfl', 'wtfr', 'wtrl', 'wtrr', 'app', 'ag', 'btd', 'drfl', 'drfr', 'drrl', 'drrr', 'ect', 'eot', 'iat', 'laig', 'lain', 'loig', 'loin', 'oap', 'tlm', 'tlr', 'tg', 'tdp', 'tl', 'tsh', 'tot', 'ttfl', 'ttfr', 'ttrl', 'ttrr', 'ttsfl', 'ttsfr', 'ttsrl', 'ttsrr', 'tpfl', 'tpfr', 'tprl', 'tpr

59 tags are used on every data point, 129 arent used consistently.

Next we extract all datapoints which only have the following features:

- all consistent ones (59)

- Also latitude and longitude = 61 

- circuitname + racenumber + lapnumber + segmentnumber = 65

- userProfileGUID (driverunique ID) + userProfileName (driver Name) + ELEVATION (can be null)

- average and top Speed + laptime = **71 Features**

Lets form a pandas dataset out of it.

In [4]:
import glob
import xml.etree.ElementTree as ET
namespace = "{http://www.topografix.com/GPX/1/1}"
root_dir = './GPX Files\\'

#data should look like this, so it can be converted to pandas dataframe:
#data = {'ts': [3, 2, 1, 0], 'dfs': ['a', 'b', 'c', 'd']}
data = {}
racenumber = -1
circuitname = ""
for filename in glob.iglob(root_dir + '**/*.gpx', recursive=True):
    
    print ('currentFile', filename)
    #if FileName is Track: Skip
    if 'Track' in str(filename):
        continue
    
    
    currentcircuitname = filename.split(root_dir)[1].split("\\")[0]
    racefilename = filename.split(root_dir)[1].split("\\")[1]

    if currentcircuitname != circuitname:
        racenumber = 1
        circuitname =currentcircuitname

    root = ET.parse(filename).getroot()
    trknumber = 0
    lapCounter=1
    #RaceDriver Id and Name
    raceMeta = root.find (namespace + "metadata").find(namespace + "extensions").find(namespace + "raceMeta")
    userProfileName = None
    userProfileGUID = None
    for amgraw in raceMeta.findall("*"):
        amg = str(amgraw).split("{https://www.mercedes-amg.com/xsd/gpx-ext}")[1].split('\'')[0]
        if amg == "userProfileName":
            userProfileName = str(amgraw.text)
        elif amg == "userProfileGUID":
            userProfileGUID = str(amgraw.text)

    for trk in root.findall(namespace + "trk"):
        #first <trk> is only trackdata and is not needed
        if trknumber > 0:
            #get AverageLapSpeed in kmh and Top Speed  in kmh + Laptime in ms
            averageLapSpeed = float(trk.find(namespace + "extensions").find(namespace + "lapAverageSpeed").text)/10
            topLapSpeed = float(trk.find(namespace + "extensions").find(namespace + "lapTopSpeed").text)/10
            lapTime = float(trk.find(namespace + "extensions").find(namespace + "lapTime").text)/1000

            first_trkpt_of_trk = True;
            segCounter = 1
            for seg in trk.findall(namespace +"trkseg"):
                for trkpt in seg.findall(namespace + "trkpt"):
                    #current_trkpt_data needs to be overwritten for each new trackpoint
                    current_trkpt_data = {}
                    #get coordinte data of trkpt(always provided)
                    current_trkpt_data['lat'] = float(trkpt.attrib['lat'])
                    current_trkpt_data['lon'] = float(trkpt.attrib['lon'])
                    current_trkpt_data['circuitname'] = str(circuitname)
                    current_trkpt_data['racenumber'] = float(racenumber)
                    current_trkpt_data['lap'] = int(lapCounter)
                    current_trkpt_data['segment'] = int(segCounter)
                    current_trkpt_data['userProfileGUID'] = userProfileGUID
                    current_trkpt_data['userProfileName'] = userProfileName
                    #elevation if available(can remain None)
                    elevation = None
                    elevationElement = trkpt.find(namespace + "ele")
                    if (elevationElement is not None):
                        elevation = elevationElement.text
                    
                    current_trkpt_data['elevation'] = elevation
                    # AverageLapSpeed in kmh and Top Speed  in kmh + Laptime in ms
                    current_trkpt_data['averageLapSpeed'] = averageLapSpeed
                    current_trkpt_data['topLapSpeed'] = topLapSpeed
                    current_trkpt_data['lapTime'] = lapTime
                    
                

                    
                    if first_trkpt_of_trk:
                        #-->just copy trkp data to current_trkpt_data
                        for extension in alltkrptsextensions:
                            value = float(trkpt.find(namespace + "extensions").find(namespace + extension).text)
                            current_trkpt_data[extension] = value
                        first_trkpt_of_trk = False
                    else:
                        #-->copy trkp data to current_trkpt_data, and get remaining data from last trkpt
                        for extension in alltkrptsextensions:
                            #check if extension is in current trkpt
                            
                            #get available extensions in trkp
                            raw_available_extensions = trkpt.find(namespace + "extensions").findall(namespace + extension)
                            available_extensions=[]
                            for raw_available_extension in raw_available_extensions:
                                available_extensions.append(str(raw_available_extension).split(namespace)[1].split("\'")[0])
                            if extension in available_extensions:
                                #if yes: copy extension data to current_trkpt_data
                                value = float(trkpt.find(namespace + "extensions").find(namespace + extension).text)
                                current_trkpt_data[extension] = value
                            else:
                                #if not: copy data from previous trkpt
                                current_trkpt_data[extension] = data[extension][-1]
                    #print(current_trkpt_data['dfs'])
                    #append current_trkpt_data to data
                    if not data:
                        for key in current_trkpt_data.keys():
                            data[key] = [current_trkpt_data[key]]
                    else:
                        for key in current_trkpt_data.keys():
                            data[key].append(current_trkpt_data[key])
                            
                    #print("circuitname", current_trkpt_data['circuitname'] )
                    #print("racenumber" ,current_trkpt_data['racenumber'] )
                    #print("lapCounter",current_trkpt_data['lap'] )
                    #print("segCounter",current_trkpt_data['segment'])

                                     
                            
                segCounter +=1
            lapCounter += 1
        trknumber+=1
    racenumber+=1



currentFile ./GPX Files\Circuit Ricardo Tormo unofficial\Race_20171204T093133.gpx
currentFile ./GPX Files\Circuit Ricardo Tormo unofficial\Race_20171205T074157.gpx
currentFile ./GPX Files\Circuit Ricardo Tormo unofficial\Race_20171205T110519.gpx
currentFile ./GPX Files\Circuit Ricardo Tormo unofficial\Race_20171205T131202.gpx
currentFile ./GPX Files\Circuit Ricardo Tormo unofficial\Race_20171207T130651.gpx
currentFile ./GPX Files\Circuit Ricardo Tormo unofficial\Race_20171208T081031.gpx
currentFile ./GPX Files\Circuit Ricardo Tormo unofficial\Race_20171209T125620.gpx
currentFile ./GPX Files\Circuit Ricardo Tormo unofficial\Race_20171209T144140.gpx
currentFile ./GPX Files\Circuit Ricardo Tormo unofficial\Race_20171209T153538.gpx
currentFile ./GPX Files\Circuit Ricardo Tormo unofficial\Race_20171209T154549.gpx
currentFile ./GPX Files\Circuit Ricardo Tormo unofficial\Race_20171210T094031.gpx
currentFile ./GPX Files\Circuit Ricardo Tormo unofficial\Race_20171210T111310.gpx
currentFile ./GP

In [5]:
# check shape of extracted data, should be 71xn
print(len(data))
for key in data.keys():
    print(len(data[key]), key)

71
265357 lat
265357 lon
265357 circuitname
265357 racenumber
265357 lap
265357 segment
265357 userProfileGUID
265357 userProfileName
265357 elevation
265357 averageLapSpeed
265357 topLapSpeed
265357 lapTime
265357 ts
265357 dfs
265357 h
265357 v
265357 atod
265357 bpdr
265357 dtf
265357 dtfl
265357 dtfr
265357 dtr
265357 dtrl
265357 dtrr
265357 epd
265357 er
265357 etd
265357 odrn
265357 sbc
265357 sbv
265357 ax
265357 ay
265357 vla
265357 vlb
265357 vsd
265357 vyr
265357 wtfl
265357 wtfr
265357 wtrl
265357 wtrr
265357 app
265357 btd
265357 drfl
265357 drfr
265357 drrl
265357 drrr
265357 ect
265357 eot
265357 iat
265357 laig
265357 lain
265357 loig
265357 loin
265357 tg
265357 tl
265357 tsh
265357 tot
265357 ttfl
265357 ttfr
265357 ttrl
265357 ttrr
265357 ttsrl
265357 tpfl
265357 tpfr
265357 tprl
265357 tprr
265357 tpms
265357 tsfr
265357 ac
265357 swa
265357 fwa


In [6]:
import pandas as pd
dataframe = pd.DataFrame.from_dict(data)

In [7]:
print(dataframe.shape)

(265357, 71)


For some tableau plots we want to use the following features:

- circuitname
- racenumber
- lap
- segment
- lapTime
- userProfileGUID
- userProfileName
- speed: vsd
- elevation
- topLapSpeed
- averageLapSpeed
- total metres driven: dfs
- time-stamp: ts
- lat
- lon

In [8]:
tableau_dataframe = dataframe[['circuitname','racenumber','lap','segment','userProfileGUID','userProfileName','lapTime','vsd','elevation', \
                                  'topLapSpeed','averageLapSpeed','dfs', 'ts', 'lat', 'lon']] 

In [9]:
tableau_dataframe.head()

,circuitname,racenumber,lap,segment,userProfileGUID,userProfileName,lapTime,vsd,elevation,topLapSpeed,averageLapSpeed,dfs,ts,lat,lon
0,Circuit Ricardo Tormo unofficial,1.0,1,1,00000000-0000-0000-0000-000000000000,Gast,191.563,1968.0,None,219.9,78.9,0.00000,0.0,39.483515,-0.631086
1,Circuit Ricardo Tormo unofficial,1.0,1,1,00000000-0000-0000-0000-000000000000,Gast,191.563,1988.0,None,219.9,78.9,5.46565,105.0,39.483543,-0.631034
2,Circuit Ricardo Tormo unofficial,1.0,1,1,00000000-0000-0000-0000-000000000000,Gast,191.563,1988.0,None,219.9,78.9,10.71300,205.0,39.483570,-0.630984
3,Circuit Ricardo Tormo unofficial,1.0,1,1,00000000-0000-0000-0000-000000000000,Gast,191.563,2007.0,None,219.9,78.9,15.97620,305.0,39.483597,-0.630934
4,Circuit Ricardo Tormo unofficial,1.0,1,1,00000000-0000-0000-0000-000000000000,Gast,191.563,2007.0,None,219.9,78.9,21.28570,405.0,39.483624,-0.630883


In [10]:
#write to File
tableau_dataframe.to_csv('secondTask_data.txt', sep=';', index=False)